In [12]:
import pandas as pd

In [13]:
df = pd.read_csv('building_data/edu.csv')
df

,POI_YM,POI_ID,POI_LON,POI_LAT,CATEGORY,POI_X,POI_Y
0,201901,18504,121.607648,25.055949,中高級校,311308.923926,2.772112e+06
1,201901,22384,121.501561,25.090958,補習補校,300590.646195,2.775946e+06
2,201901,43092,121.551987,25.100629,幼教,305672.654242,2.777037e+06
3,201901,28917,121.526849,25.119752,補習補校,303128.957148,2.779145e+06
4,201901,18085,121.523447,25.087360,中高級校,302799.790653,2.775556e+06
...,...,...,...,...,...,...,...
9822,202401,30136,121.586206,25.038968,補習補校,309153.621504,2.770222e+06
9823,202401,25266,121.574865,25.045367,補習補校,308006.174072,2.770926e+06
9824,202401,34507,121.576631,25.038511,補習補校,308187.609575,2.770167e+06
9825,202401,43159,121.570079,25.041244,幼教,307525.160747,2.770467e+06


In [14]:
print(df['CATEGORY'].unique())

['中高級校' '補習補校' '幼教' '社區大學' '研究機構' '大專院校' '特教' '外僑學校' '軍警校院']


In [15]:
# label encode 
df['CATEGORY_CODE'] = df['CATEGORY'].astype('category')
df['CATEGORY_CODE'] = df['CATEGORY_CODE'].cat.codes
df

,POI_YM,POI_ID,POI_LON,POI_LAT,CATEGORY,POI_X,POI_Y,CATEGORY_CODE
0,201901,18504,121.607648,25.055949,中高級校,311308.923926,2.772112e+06,0
1,201901,22384,121.501561,25.090958,補習補校,300590.646195,2.775946e+06,7
2,201901,43092,121.551987,25.100629,幼教,305672.654242,2.777037e+06,3
3,201901,28917,121.526849,25.119752,補習補校,303128.957148,2.779145e+06,7
4,201901,18085,121.523447,25.087360,中高級校,302799.790653,2.775556e+06,0
...,...,...,...,...,...,...,...,...
9822,202401,30136,121.586206,25.038968,補習補校,309153.621504,2.770222e+06,7
9823,202401,25266,121.574865,25.045367,補習補校,308006.174072,2.770926e+06,7
9824,202401,34507,121.576631,25.038511,補習補校,308187.609575,2.770167e+06,7
9825,202401,43159,121.570079,25.041244,幼教,307525.160747,2.770467e+06,3


In [16]:
p_taipei = pd.read_csv('points_taipei.csv')
p_taipei

,x1,y1,x2,y2
0,296103,2777535,296203,2777635
1,296103,2777635,296203,2777735
2,296103,2777735,296203,2777835
3,296103,2777835,296203,2777935
4,296103,2777935,296203,2778035
...,...,...,...,...
27656,317103,2768935,317203,2769035
27657,317103,2769035,317203,2769135
27658,317103,2769135,317203,2769235
27659,317103,2769235,317203,2769335


In [17]:
p_new_taipei = pd.read_csv('points_new_taipei.csv')
p_new_taipei

,x1,y1,x2,y2
0,278508,2777870,278608,2777970
1,278508,2777970,278608,2778070
2,278508,2778070,278608,2778170
3,278508,2778170,278608,2778270
4,278508,2778370,278608,2778470
...,...,...,...,...
209743,351508,2767770,351608,2767870
209744,351608,2766970,351708,2767070
209745,351608,2767070,351708,2767170
209746,351608,2767270,351708,2767370


In [18]:
# 分別計算每個方格內不同種類學校的數量
def count_category(df, p):
    count = [(df[(df['POI_Y'].between(p['y1'][i], p['y2'][i])) & (df['POI_X'].between(p['x1'][i], p['x2'][i]))]).shape[0] for i in range(len(p))]

    count_list = []
    for i in range(len(p)):
        condition = (df['POI_Y'].between(p['y1'][i], p['y2'][i])) & (df['POI_X'].between(p['x1'][i], p['x2'][i]))
        count_list.append([((df[condition & (df['CATEGORY_CODE'] == j)])).shape[0] for j in range(df['CATEGORY'].nunique())])

    edu_id = [(df[(df['POI_Y'].between(p['y1'][i], p['y2'][i])) & (df['POI_X'].between(p['x1'][i], p['x2'][i]))]['POI_ID']).values for i in range(len(p))]

    return count, count_list, edu_id

In [19]:
#呼叫函數
p_taipei["count"], p_taipei["count_list"], p_taipei["edu_id"] = count_category(df, p_taipei)
p_new_taipei["count"], p_new_taipei["count_list"], p_new_taipei["edu_id"] = count_category(df, p_new_taipei)

In [20]:
p_taipei.sort_values(by='count', ascending=False)

,x1,y1,x2,y2,count,count_list,edu_id
5059,301903,2770835,302003,2770935,19,"[0, 0, 0, 0, 0, 0, 0, 19, 0]","[24149, 24425, 24306, 34466, 25191, 36841, 230..."
16147,306903,2771535,307003,2771635,13,"[0, 0, 0, 0, 0, 0, 0, 13, 0]","[30120, 35934, 27764, 36073, 25699, 36577, 351..."
5226,302003,2770835,302103,2770935,13,"[0, 0, 0, 0, 0, 0, 0, 13, 0]","[24368, 32261, 23891, 36354, 34120, 20919, 343..."
7650,303303,2779035,303403,2779135,13,"[0, 0, 0, 0, 0, 0, 0, 13, 0]","[26352, 27039, 31596, 32612, 24705, 27792, 352..."
12265,305403,2774535,305503,2774635,12,"[0, 0, 0, 1, 0, 0, 0, 11, 0]","[34319, 32754, 19651, 28650, 21865, 23043, 228..."
...,...,...,...,...,...,...,...
9682,304303,2770735,304403,2770835,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",[]
9681,304303,2770635,304403,2770735,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",[]
9678,304303,2770335,304403,2770435,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",[]
9677,304303,2770235,304403,2770335,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",[]


In [21]:
p_taipei = pd.concat([p_taipei, p_taipei['count_list'].apply(pd.Series)], axis=1)
p_taipei

,x1,y1,x2,y2,count,count_list,edu_id,0,1,2,3,4,5,6,7,8
0,296103,2777535,296203,2777635,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",[],0,0,0,0,0,0,0,0,0
1,296103,2777635,296203,2777735,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",[],0,0,0,0,0,0,0,0,0
2,296103,2777735,296203,2777835,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",[],0,0,0,0,0,0,0,0,0
3,296103,2777835,296203,2777935,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",[],0,0,0,0,0,0,0,0,0
4,296103,2777935,296203,2778035,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",[],0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27656,317103,2768935,317203,2769035,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",[],0,0,0,0,0,0,0,0,0
27657,317103,2769035,317203,2769135,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",[],0,0,0,0,0,0,0,0,0
27658,317103,2769135,317203,2769235,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",[],0,0,0,0,0,0,0,0,0
27659,317103,2769235,317203,2769335,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",[],0,0,0,0,0,0,0,0,0


In [22]:
p_new_taipei = pd.concat([p_new_taipei, p_new_taipei['count_list'].apply(pd.Series)], axis=1)
p_new_taipei

,x1,y1,x2,y2,count,count_list,edu_id,0,1,2,3,4,5,6,7,8
0,278508,2777870,278608,2777970,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",[],0,0,0,0,0,0,0,0,0
1,278508,2777970,278608,2778070,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",[],0,0,0,0,0,0,0,0,0
2,278508,2778070,278608,2778170,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",[],0,0,0,0,0,0,0,0,0
3,278508,2778170,278608,2778270,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",[],0,0,0,0,0,0,0,0,0
4,278508,2778370,278608,2778470,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",[],0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209743,351508,2767770,351608,2767870,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",[],0,0,0,0,0,0,0,0,0
209744,351608,2766970,351708,2767070,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",[],0,0,0,0,0,0,0,0,0
209745,351608,2767070,351708,2767170,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",[],0,0,0,0,0,0,0,0,0
209746,351608,2767270,351708,2767370,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",[],0,0,0,0,0,0,0,0,0


In [23]:
p_taipei = p_taipei.drop(columns=['count_list'])
p_new_taipei = p_new_taipei.drop(columns=['count_list'])

category_map = dict(zip(df['CATEGORY_CODE'].unique(), df['CATEGORY'].unique()))

p_taipei = p_taipei.rename(columns=category_map)
p_new_taipei = p_new_taipei.rename(columns=category_map)

In [24]:
# 寫入csv
p_taipei.to_csv("points_taipei_edu.csv", index=False)
p_new_taipei.to_csv("points_new_taipei_edu.csv", index=False)